# TME 5 : Projet Detection de motifs


## Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [1]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence



def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    nucList = [nuc[random.randint(0,3)].upper() if upper else nuc[random.randint(0,3)].lower() for i in range(n)]
    sequence = "".join(nucList)
    return sequence
testMotif = generateRandomSequence(6)
print(testMotif)

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = motif
    posToModified = random.sample(range(len(motif)), nbpos)
    for pos in posToModified:
        newNuc = random.choice(nuc)
        while newNuc == motif[pos]:
            newNuc = random.choice(nuc)
        motifM = motifM[0:pos] + newNuc + motifM[pos+1:]
    return motifM


testMotif = modifierMotif(testMotif, 1)
print(testMotif)

def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
    motif = generateRandomSequence(k)
    modifiedMotifs = [modifierMotif(motif, v) for i in range(t)]
    posToInsert = [random.randint(0, n-k-1) for i in range(t)]
    preSequences = [generateRandomSequence(n-k) for i in range(t)]
    sequences = [preSequences[i][:posToInsert[i]] + modifiedMotifs[i] + preSequences[i][posToInsert[i]+1:] for i in range(t)]
    return sequences

adn = implantMotifVar(k, v, t, n)
print (adn)


GTCATG
GTAATG
['CGTCGATGC', 'ATATTCGAG', 'TAGTTGCAA']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [2]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
print (adn)

['GAGACCCGG', 'GATTACCGA', 'AGAGCCCCG', 'GTGGGCCGA', 'GAGTGAGGA', 'GAGTTCTGC', 'CGTGTGCCA', 'AAGTGCGGT', 'TAAGTCCCG', 'AGGGTGTCG']


3\. Préparation pour l'algorithme _"Brute Force Motif Search"_. Avant d'implementer  l'algorithme _"Brute Force Motif Search"_, faites une fonction pour calculer le score `score(s, seqs)`, où `s` est un ensemble de position initiales `s`$=(s_1,s_2,...,s_k)$  et `seqs` est une matrice de taille `t x n`  (`t` est le nombre de séquences et `n` la taille de ces séquences), pour tester votre fonction utiliser la matrice et les valeurs ci-dessous (voir aussi les slides du cours).


In [3]:
import numpy as np

k =8
n = 68 # length of each DNA sequence
t = 5 # number of  DNA sequence samples

s=(25,20,2,55,59)

adn = [
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn avec les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs
    """
    # motifs = [seqs[i][s[i]:s[i]+k] for i in range(len(seqs))]    
    motifs = []
    for i in range(len(seqs)):
        try:
            motifs.append(seqs[i][s[i]:s[i]+k])
        except IndexError:
            pass
    return motifs


def  profile(motifs, k, nuc):
    """
    Construire une PWM (position weight matrix)
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie PWM: position weight matrix
    """
    nDict = {nuc[i]: i for i in range(len(nuc))}
    q = len(nuc)
    PWM = np.zeros((q, k))
    for col in range(k):
        for line in range(len(motifs)):
            # try:
            nucleotide = motifs[line][col]
            # except IndexError:
                # print(line, col, motifs)
            PWM[nDict[nucleotide]][col] += 1
    return PWM


def score(prof, k):
    """
    Calcul le score d'un PWM, la somme de max de chaque colonne
    entrée PWM: position weight matrix
    entrée k: taille du motif
    sortie score: la somme de max de chaque colonne
    """
    sc = 0
    for col in range(k):
        sc += max(prof[:,col])
    return int(sc)

motifs = extractSeqs(s, adn, k)
print(motifs)
PWM = profile(motifs, k, nuc)
print(PWM)

sc = score(PWM, k)
#teste, sc doit etre egalle à 30
print (sc)

['AGGTACTT', 'CCATACGT', 'ACGTTAGT', 'ACGTCCAT', 'CCGTACGG']
[[3. 0. 1. 0. 3. 1. 1. 0.]
 [2. 4. 0. 0. 1. 4. 0. 0.]
 [0. 1. 4. 0. 0. 0. 3. 1.]
 [0. 0. 0. 5. 1. 0. 1. 4.]]
30


4\. Implementer l'algoritme _"Brute Force Motif Search"_ pour chercher des motifs de taille variable. Attention, la complexité de cet algorithme est trop importante, car il enumère toutes les positions intiales possibles, nous allons donc le tester seulement sur de petits jeux de données.

In [4]:
from itertools import product

#enumeration all initial positions
def initialPositions(k, n, t):
    """
    Génère des vecteurs avec tous les positions de départ possible
    entrée k: taille du motif
    entrée n : longueur des séquences 
    entrée t : nombre de séquences 
    """
    listPos = list(range(n-k))
    allIniPos = product(listPos, repeat=t)
    return allIniPos

def bruteForceMotifSearch(sequences, t, n, k):
    """
    Implémente l'algorithme bruteForceMotifSearch 
    entrée séquences: matrice de dimension txn avec les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k: taille du motif 
    sortie bestMotif: vecteur de positions de départ contenant le motif de meilleur score 
    sortie bestscore: score associé au meilleur motif
    """
    bestscore = 0; bestMotif=''
    allIniPos = initialPositions(k, n, t)
    for s in allIniPos:
        scTmp = score(profile(extractSeqs(s, sequences, k), k, nuc), k)
        if scTmp > bestscore:
            bestscore = scTmp
            bestMotif = s
            # print(s, scTmp, extractSeqs(s, sequences, k))
    return bestMotif, bestscore

#test
k=4; v=1; t=3; n=10
sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
print (bestMotif, bestscore)
print (extractSeqs(bestMotif, sequences, k))


['CGCTTCCTA', 'TCTTGATTA', 'GATGATTTG']
(1, 4, 3) 11
['GCTT', 'GATT', 'GATT']


5\. L'algorithme est-il performant sur ce jeu de données ? Sinon, pourquoi ?

<font color='blue'>
reponse: Oui.
</font>

6\. Tester l'agorithme avec les paramètres suivants `k=7`, `v=1`, `t=5`, et `n=20`. Avez vous de meilleurs performances ?

In [5]:
k=7; v=1; t=5; n=20

sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)

print (bestMotif)
print (extractSeqs(bestMotif, sequences, k))

['CTCCGCGGGCATCCTTTCG', 'CTTCCGATATCCCTAAGAC', 'AGTAGTGGTATATCCATAC', 'TTCATATCCTGTGAATAGA', 'GGCAAGGTTATCCTAGCGC']
(7, 5, 8, 2, 6)
['GGCATCC', 'GATATCC', 'TATATCC', 'CATATCC', 'GTTATCC']


7\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les mêmes jeux de données que la question 3 (voir slides de cours). Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles.

In [6]:
#Genere tous les K-mers de taille K ayant de AAA... à TTT...
allkmers = product(nuc, repeat=k)

def kmerList(allKmers):
    return ["".join(nucTuple) for nucTuple in allKmers]

kmers = kmerList(allkmers)

#Enlever les motifs peu complexe

repeatMax = 5
simpleRepeatMax = 4

def isRepeatN_2(seq, repeatMax):
    repeatDict = {nu: 0 for nu in nuc}
    for nu in seq:
        repeatDict[nu] += 1
        if repeatDict[nu] >= 5:
            return True
    return False

def isSimpleRepeat_(seq, simpleRepeatMax):
    if len(seq) <= 3:
        return False
    a = seq[0]
    b = seq[1]
    repeat = "".join([a if i%2 == 0 else b for i in range(simpleRepeatMax)])
    return seq.find(repeat) != -1

def isRepeatN(seq):
    return isRepeatN_2(seq, repeatMax)
    
def isSimpleRepeat(seq):
    return isSimpleRepeat_(seq, simpleRepeatMax)

def removeLowComplexePair(kmers):
    return [kmer for kmer in kmers if not isSimpleRepeat(kmer) and not isRepeatN(kmer)]

kmersValid = removeLowComplexePair(kmers)
print (len(kmers))
print (len(kmersValid))
# print(kmersValid)

16384
14640


In [7]:
def hamdist(str1, str2):
    """
    Calcul la distance de hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de hamming
    """
    assert(len(str1) == len(str2))
    return sum([1 if str1[i]!=str2[i] else 0 for i in range(len(str1))])

assert(hamdist("acgcga", "acgcgg") == 1)

def totalDistance(motif, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: sequence consensus
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée k: taille du motif 
    sortie td: somme de distance de hamming minimal
    """
    assert(len(motif) == k)
    tc = 0
    for i in range(len(sequences)):
        sequence = sequences[i]
        lenSeq = len(sequence)
        distList = [hamdist(motif, sequence[j: j+k]) for j in range(lenSeq-k+1)]
        tc += min(distList)
    return tc
assert(totalDistance("acg", ["acga", "actt"], 3) == 1)



def MedianStringSearch(allkmers, sequences, t, n, k):
    """
    Implement l'algorithme MedianStringSearch
    entrée allkmers: liste de K-mers valides
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée n : longueur des séquences 
    entrée t : nombre de séquences 
    entrée k: taille du motif 
    sortie bestMotif: le motif que minimise les distances
    sortie bestDistance: la distance minimal
    sortie motifDist: un dictionnaire contenant les motifs et leurs distances
    REMARK: n peut varié dans les fichiers de chip-seq, vous pouvez donc l'extraire de matrice sequence 
    """
    bestDistance = 1000
    bestMotif = ''
    motifDist = {}
    for kmer in allkmers:
        distTmp = totalDistance(kmer, sequences, k)
        if distTmp < bestDistance:
            bestMotif, bestDistance = kmer, distTmp
        motifDist[kmer] = distTmp
    return bestMotif, bestDistance, motifDist

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, sequences, t, n, k)

# print(bestMotif, bestDistance, motifDist)

In [8]:
#Apliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

#Faire un merge de motifs trouvés dans les deux brin

def sortDictMotifToList(motifDict : dict):
    return sorted(motifDict.items(), key=lambda item:item[1], reverse=True)

def mergeMotifs(motifs, motifsRevComp):
    ''' 
    Merge motifs trouvé dans les brins sens et complementaire
    entrée motifs : un dictionnaire contenant les motifs du brin sens et leurs distances
    entrée motifsRevComp : un dictionnaire contenant les motifs du brin complementaire et leurs distances
    sortie allMotif: merge de motifs et motifsRevComp
    '''
    a = set(motifs.keys())
    b = set(motifsRevComp.keys())
    keyUnion = a.union(b)
    dict3 = {motif: 0 for motif in keyUnion}
    dict3 = {motif: (dict3[motif] + motifs[motif]) for motif in motifs.keys()}
    dict3 = {motif: (dict3[motif] + motifsRevComp[motif]) for motif in motifsRevComp.keys()}
    return dict3


#Faire une fonction pour afficher les top meilleurs motifs (distance minimal)

def printTopFMotifsFreq(motifDist, top, rev=False):
    """
    Afficher les top motifs avec les plus petit distances
    entrée motifDist: un dictionnaire contenant les motifs et leurs distances
    entrée top: max de motifs à afficher
    sortie : None
    """
    motifsRet = {}
    res = sortDictMotifToList(motifDist)
    if rev:
        res = res[::-1]
    print(res[:top])

def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée : sequence de nucléotides (brin sens)
    sortie : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    res = ''
    for index in range(len(seq)):
        res += compl[seq[index].upper()]
    return res[::-1]

def reverseSequences(sequences):
    """
    Trouver les reverses complémentaires pour une liste de sequnces
    entrée sequences: une liste de sequnces
    sortie : une liste de sequnces dont les éléments sont reverses complémentaires de celui d'entrée
    """
    return [reversecompl(seq) for seq in sequences]

printTopFMotifsFreq(motifDist, 6, True)
revSequences = "" #sequences du brin complementaire 

revSequences = reverseSequences(sequences)


bestMotifRev, bestDistanceRev, motifDistRev = MedianStringSearch(kmersValid, revSequences, t, n, k)
print (bestMotifRev, bestDistanceRev)

printTopFMotifsFreq(motifDistRev, 6, True)

allmotif = mergeMotifs(motifDist, motifDistRev)
print ('mergerd')
printTopFMotifsFreq(allmotif, 6, True)

[('TATCCTT', 5), ('GATATCC', 5), ('TTATCCT', 7), ('TATCCTG', 7), ('TATCCTA', 7), ('TATCCCT', 7)]
AAGGATA 5
[('GGATATC', 5), ('AGGATAT', 5), ('AAGGATA', 5), ('TGGATAT', 7), ('TAGGATA', 7), ('TAAGGAT', 7)]
mergerd
[('GGATATC', 12), ('GATATCC', 12), ('AGGATAT', 16), ('TGGATAT', 18), ('GGGATAT', 18), ('GGATATA', 18)]


8\. Comparation entre les algorithmes _"Median String Search"_ et "_Brute Force Motif Search_"

a\. Produire des séquences artificielles avec les paramètres suivants  `k=7`, `v=1`, `t=5`, `n=20`. Comparer les performance et le temps de calculs des deux algorithmes.

In [9]:
k=7; v=1; t=5; n=20
motifVar = implantMotifVar(k, v, t, n)
print(motifVar)

['GCATTAAGTTTCAACGGTT', 'CCAAGATACCGGTGGGGAC', 'CATTCACCCGTCGGGACCC', 'CACCCGTGTTGCATGGGAG', 'CACGCCGGTAACAGTTGTT']


In [10]:
s, bestscore = bruteForceMotifSearch(motifVar, t, n, k)

In [11]:
print(s, extractSeqs(s, motifVar, k), bestscore)

(11, 6, 4, 0, 2) ['CAACGGT', 'TACCGGT', 'CACCCGT', 'CACCCGT', 'CGCCGGT'] 30


In [12]:
allkmers = product(nuc, repeat=k)
kmersValid = removeLowComplexePair(kmerList(allkmers))
MedianStringSearch(kmersValid, motifVar, t, n, k)

('CACCGGT',
 5,
 {'AAACACC': 16,
  'AAACACG': 17,
  'AAACACT': 15,
  'AAACAGC': 15,
  'AAACAGG': 15,
  'AAACAGT': 12,
  'AAACATC': 16,
  'AAACATG': 17,
  'AAACATT': 16,
  'AAACCAC': 18,
  'AAACCAG': 16,
  'AAACCAT': 15,
  'AAACCCA': 18,
  'AAACCCC': 18,
  'AAACCCG': 15,
  'AAACCCT': 15,
  'AAACCGA': 14,
  'AAACCGC': 14,
  'AAACCGG': 12,
  'AAACCGT': 10,
  'AAACCTA': 19,
  'AAACCTC': 17,
  'AAACCTG': 15,
  'AAACCTT': 15,
  'AAACGAC': 17,
  'AAACGAG': 17,
  'AAACGAT': 16,
  'AAACGCA': 19,
  'AAACGCC': 17,
  'AAACGCG': 16,
  'AAACGCT': 16,
  'AAACGGA': 15,
  'AAACGGC': 15,
  'AAACGGG': 13,
  'AAACGGT': 11,
  'AAACGTA': 15,
  'AAACGTC': 15,
  'AAACGTG': 14,
  'AAACGTT': 13,
  'AAACTAC': 18,
  'AAACTAG': 18,
  'AAACTAT': 17,
  'AAACTCA': 20,
  'AAACTCC': 18,
  'AAACTCG': 17,
  'AAACTCT': 18,
  'AAACTGA': 17,
  'AAACTGC': 17,
  'AAACTGG': 16,
  'AAACTGT': 13,
  'AAACTTA': 17,
  'AAACTTC': 15,
  'AAACTTG': 15,
  'AAACTTT': 17,
  'AAAGACC': 18,
  'AAAGACG': 18,
  'AAAGACT': 18,
  'AAAGAGC': 18

b\. Quel algorithme peut etre appliqué au probleme de recherche de motifs? Et comment?

In [13]:
n = 100
# n = 100
t = 100

def readFasta(fastaFile, n):
    """
    Lire n sequences d'un ficher fasta
    entrée fastaFile: nom du fichier
    entrée n: nombre de sequences à retourner
    sortie sequence: liste de sequences
    """
    sequence = []
    file = open(fastaFile, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence

peaksFile = "Sequence_by_Peaks_2.fasta" #votre fichier de Peaks
sequences   = readFasta(peaksFile, t)


<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'Sequence_by_Peaks_2.fasta'

In [ ]:
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, len(sequences[0]), k)
print(bestMotif, bestscore)
print(extractSeqs(bestMotif, sequences, k))

In [ ]:
allkmers = product(nuc, repeat=k)
kmersValid = removeLowComplexePair(kmerList(allkmers))
resMS = MedianStringSearch(kmersValid, sequences, t, len(sequences[0]), k)
print(resMS[0:2])